In [1]:
import requests
import pandas as pd
import pickle
from time import sleep
mainpage = 'https://www.immobiliare.it'

In [2]:
from time import time

In [6]:
def get_announcement(ann):
    title = ann.find('p', class_ = 'titolo text-primary').a['title']
    link = ann.find('p', class_ = 'titolo text-primary').a['href']

    locali, superficie, bagni, piano = 'NA','NA','NA','NA'

    if not link.startswith('https://www.immobiliare.it'):
        link = mainpage + link
    subcontent = requests.get(link)
    subsoup = BeautifulSoup(subcontent.text, "html.parser")
    infos = subsoup.find('div', class_='im-property__features')


    price = infos.find('span', class_='features__price--double')
    if price is None:
        price = infos.find('ul', class_='features__price-block').find('li', class_='features__price').span
    price = price.getText()

    subinfos = infos.find('ul', class_='list-inline list-piped features__list')


    for list_item in subinfos.find_all('li'):
        feature = list_item.find('div', class_='features__label')

        if feature is not None:
            if feature.getText() == 'locali':
                locali = list_item.find('div').find('span').getText()
            elif feature.getText() == 'superficie':
                superficie = list_item.find('div').find('span').getText()
            elif feature.getText() == 'bagni':
                bagni = list_item.find('div').find('span').getText()
            elif feature.getText() == 'piano':
                
                piano = list_item.find('div').find('span')
                if piano is None:
                    piano = list_item.find('div').find('abbr')
                    
                piano = piano.getText()


    description = subsoup.find('div', class_='left-side')
    description = description.find('div', class_='col-xs-12 description-text text-compressed').div.getText()

    attr_list=[title, price, locali, superficie, bagni, piano, description]
    sleep(1)
    return attr_list

In [7]:
from time import time

In [8]:
from bs4 import BeautifulSoup

def scraping_function():
    counter=0

    ann_list = []

    for i in range(1,1700):

        # request the html page
        content = requests.get("https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=%i"%i)
        # soupify it
        soup = BeautifulSoup(content.text, "lxml")


        # find all the tags li with class 'listing-item vetrina js-row-detail'
        announcements = soup.find_all('li', class_ = 'listing-item vetrina js-row-detail')
        if len(announcements) == 0:
            announcements = soup.find_all('li', class_ = 'listing-item js-row-detail')


        for ann in announcements:

            try:
                attr_list = get_announcement(ann)
                ann_list.append(attr_list)
                counter=counter+1
                print(i,counter)
            except:
                print('error in scraping announcement, moving to next one')
                
            columns = ['title', 'price', 'locali', 'superficie', 'bagni', 'piano', 'description']
            df = pd.DataFrame(ann_list,columns=columns)
            df.to_csv('data/raw_data.csv')

            if counter==15000:
                
                columns = ['title', 'price', 'locali', 'superficie', 'bagni', 'piano', 'description']
                df = pd.DataFrame(ann_list,columns=columns)
                df.to_csv('data/raw_data.csv')
                return



In [9]:
start = time()
df = scraping_function()
end = time()
end - start

1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
2 26
2 27
2 28
2 29
2 30
2 31
2 32
2 33
2 34
2 35
2 36
2 37
2 38
2 39
2 40
2 41
2 42
2 43
2 44
2 45
2 46
2 47
2 48
2 49
2 50
3 51
3 52
3 53
3 54
3 55
3 56
3 57
3 58
3 59
3 60
3 61
3 62
3 63
3 64
3 65
3 66
3 67
3 68
3 69
3 70
3 71
3 72
3 73
3 74
3 75
4 76
4 77
4 78
4 79
4 80
4 81
4 82
4 83
4 84
4 85
4 86
4 87
4 88
4 89
4 90
4 91
4 92
4 93
4 94
4 95
4 96
4 97
4 98
4 99
4 100
5 101
5 102
5 103
5 104
5 105
5 106
5 107
5 108
5 109
5 110
5 111
5 112
5 113
5 114
5 115
5 116
5 117
5 118
5 119
5 120
5 121
5 122
5 123
5 124
5 125
6 126
6 127
6 128
6 129
6 130
6 131
6 132
6 133
6 134
6 135
6 136
6 137
6 138
6 139
6 140
6 141
6 142
6 143
6 144
6 145
6 146
6 147
6 148
6 149
6 150
7 151
7 152
7 153
7 154
7 155
7 156
7 157
7 158
7 159
7 160
7 161
7 162
7 163
7 164
7 165
7 166
7 167
7 168
7 169
7 170
7 171
7 172
7 173
7 174
7 175
8 176
8 177
8 178
8 179
8 180
8 181
8 182
8 183
8 184
8 18

48 1192
48 1193
48 1194
48 1195
48 1196
48 1197
48 1198
48 1199
48 1200
49 1201
49 1202
49 1203
49 1204
49 1205
49 1206
49 1207
49 1208
49 1209
49 1210
49 1211
49 1212
49 1213
49 1214
49 1215
49 1216
49 1217
49 1218
49 1219
49 1220
49 1221
49 1222
49 1223
49 1224
49 1225
50 1226
50 1227
50 1228
50 1229
50 1230
50 1231
50 1232
50 1233
50 1234
50 1235
50 1236
50 1237
50 1238
50 1239
50 1240
50 1241
50 1242
50 1243
50 1244
50 1245
50 1246
50 1247
50 1248
50 1249
50 1250
51 1251
51 1252
51 1253
51 1254
51 1255
51 1256
51 1257
51 1258
51 1259
51 1260
51 1261
51 1262
51 1263
51 1264
51 1265
51 1266
51 1267
51 1268
51 1269
51 1270
51 1271
51 1272
51 1273
51 1274
51 1275
52 1276
52 1277
52 1278
52 1279
52 1280
52 1281
52 1282
52 1283
52 1284
52 1285
52 1286
52 1287
52 1288
52 1289
52 1290
52 1291
52 1292
52 1293
52 1294
52 1295
52 1296
52 1297
52 1298
52 1299
52 1300
53 1301
53 1302
53 1303
53 1304
53 1305
53 1306
53 1307
53 1308
53 1309
53 1310
53 1311
53 1312
53 1313
53 1314
53 1315
53 1316


89 2217
89 2218
89 2219
89 2220
89 2221
89 2222
89 2223
89 2224
89 2225
90 2226
90 2227
90 2228
90 2229
90 2230
90 2231
90 2232
90 2233
90 2234
90 2235
90 2236
90 2237
90 2238
90 2239
90 2240
90 2241
90 2242
90 2243
90 2244
90 2245
90 2246
90 2247
90 2248
90 2249
90 2250
91 2251
91 2252
91 2253
91 2254
91 2255
91 2256
91 2257
91 2258
91 2259
91 2260
91 2261
91 2262
91 2263
91 2264
91 2265
91 2266
91 2267
91 2268
91 2269
91 2270
91 2271
91 2272
91 2273
91 2274
91 2275
92 2276
92 2277
92 2278
92 2279
92 2280
92 2281
92 2282
92 2283
92 2284
92 2285
92 2286
92 2287
92 2288
92 2289
92 2290
92 2291
92 2292
92 2293
92 2294
92 2295
92 2296
92 2297
92 2298
92 2299
92 2300
93 2301
93 2302
93 2303
93 2304
93 2305
93 2306
93 2307
93 2308
93 2309
93 2310
93 2311
93 2312
93 2313
93 2314
93 2315
93 2316
93 2317
93 2318
93 2319
93 2320
93 2321
93 2322
93 2323
93 2324
93 2325
94 2326
94 2327
94 2328
94 2329
94 2330
94 2331
94 2332
94 2333
94 2334
94 2335
94 2336
94 2337
94 2338
94 2339
94 2340
94 2341


127 3157
127 3158
127 3159
127 3160
127 3161
127 3162
127 3163
127 3164
127 3165
127 3166
127 3167
127 3168
127 3169
127 3170
127 3171
127 3172
127 3173
127 3174
127 3175
128 3176
128 3177
128 3178
128 3179
128 3180
128 3181
128 3182
128 3183
128 3184
128 3185
128 3186
128 3187
128 3188
128 3189
128 3190
128 3191
128 3192
128 3193
128 3194
128 3195
128 3196
128 3197
128 3198
128 3199
128 3200
129 3201
129 3202
129 3203
129 3204
129 3205
129 3206
129 3207
129 3208
129 3209
129 3210
129 3211
129 3212
129 3213
129 3214
129 3215
129 3216
129 3217
129 3218
129 3219
129 3220
129 3221
129 3222
129 3223
129 3224
129 3225
130 3226
130 3227
130 3228
130 3229
130 3230
130 3231
130 3232
130 3233
130 3234
130 3235
130 3236
130 3237
130 3238
130 3239
130 3240
130 3241
130 3242
130 3243
130 3244
130 3245
130 3246
130 3247
130 3248
130 3249
130 3250
131 3251
131 3252
131 3253
131 3254
131 3255
131 3256
131 3257
131 3258
131 3259
131 3260
131 3261
131 3262
131 3263
131 3264
131 3265
131 3266
131 3267
1

163 4068
163 4069
163 4070
163 4071
163 4072
163 4073
163 4074
163 4075
164 4076
164 4077
164 4078
164 4079
164 4080
164 4081
164 4082
164 4083
164 4084
164 4085
164 4086
164 4087
164 4088
164 4089
164 4090
164 4091
164 4092
164 4093
164 4094
164 4095
164 4096
164 4097
164 4098
164 4099
164 4100
165 4101
165 4102
165 4103
165 4104
165 4105
165 4106
165 4107
165 4108
165 4109
165 4110
165 4111
165 4112
165 4113
165 4114
165 4115
165 4116
165 4117
165 4118
165 4119
165 4120
165 4121
165 4122
165 4123
165 4124
165 4125
166 4126
166 4127
166 4128
166 4129
166 4130
166 4131
166 4132
166 4133
166 4134
166 4135
166 4136
166 4137
166 4138
166 4139
166 4140
166 4141
166 4142
166 4143
166 4144
166 4145
166 4146
166 4147
166 4148
166 4149
166 4150
167 4151
167 4152
167 4153
167 4154
167 4155
167 4156
167 4157
167 4158
167 4159
167 4160
167 4161
167 4162
167 4163
167 4164
167 4165
167 4166
167 4167
167 4168
167 4169
167 4170
167 4171
167 4172
167 4173
167 4174
167 4175
168 4176
168 4177
168 4178
1

1633 4949
1633 4950
1633 4951
1633 4952
1633 4953
1634 4954
1634 4955
1634 4956
1634 4957
1634 4958
1634 4959
1634 4960
1634 4961
1634 4962
1634 4963
1634 4964
1634 4965
1634 4966
1634 4967
1634 4968
1634 4969
1634 4970
1634 4971
1634 4972
1634 4973
1634 4974
1634 4975
1634 4976
1634 4977
1634 4978
1635 4979
1635 4980
1635 4981
1635 4982
1635 4983
1635 4984
1635 4985
1635 4986
1635 4987
1635 4988
1635 4989
1635 4990
1635 4991
1635 4992
1635 4993
1635 4994
1635 4995
1635 4996
1635 4997
1635 4998
1635 4999
1635 5000
1635 5001
1635 5002
1635 5003
1636 5004
1636 5005
1636 5006
1636 5007
1636 5008
1636 5009
1636 5010
1636 5011
1636 5012
1636 5013
1636 5014
1636 5015
1636 5016
1636 5017
1636 5018
1636 5019
1636 5020
1636 5021
1636 5022
1636 5023
1636 5024
1636 5025
1636 5026
1636 5027
1636 5028
1637 5029
1637 5030
1637 5031
1637 5032
1637 5033
1637 5034
1637 5035
1637 5036
1637 5037
1637 5038
1637 5039
1637 5040
1637 5041
1637 5042
1637 5043
1637 5044
1637 5045
1637 5046
1637 5047
1637 5048


1666 5769
1666 5770
1666 5771
1666 5772
1666 5773
1666 5774
1666 5775
1666 5776
1666 5777
1666 5778
1667 5779
1667 5780
1667 5781
1667 5782
1667 5783
1667 5784
1667 5785
1667 5786
1667 5787
1667 5788
1667 5789
1667 5790
1667 5791
1667 5792
1667 5793
1667 5794
1667 5795
1667 5796
1667 5797
1667 5798
1667 5799
1667 5800
1667 5801
1667 5802
1667 5803
1668 5804
1668 5805
1668 5806
1668 5807
1668 5808
1668 5809
1668 5810
1668 5811
1668 5812
1668 5813
1668 5814
1668 5815
1668 5816
1668 5817
1668 5818
1668 5819
1668 5820
1668 5821
1668 5822
1668 5823
1668 5824
1668 5825
1668 5826
1668 5827
1668 5828
1669 5829
1669 5830
1669 5831
1669 5832
1669 5833
1669 5834
1669 5835
1669 5836
1669 5837
1669 5838
1669 5839
1669 5840
1669 5841
1669 5842
1669 5843
1669 5844
1669 5845
1669 5846
1669 5847
1669 5848
1669 5849
1669 5850
1669 5851
1669 5852
1669 5853
1670 5854
1670 5855
1670 5856
1670 5857
1670 5858
1670 5859
1670 5860
1670 5861
1670 5862
1670 5863
1670 5864
1670 5865
1670 5866
1670 5867
1670 5868


1699 6589
1699 6590
1699 6591
1699 6592
1699 6593
1699 6594
1699 6595
1699 6596
1699 6597
1699 6598
1699 6599
1699 6600
1699 6601
1699 6602
1699 6603


13909.647184848785

In [13]:
(end-start)/3600
len(pd.read_csv('data/raw_data.csv'))

6603

In [272]:

cols=['title', 'price', 'locali', 'superficie', 'bagni', 'piani', 'description']
df = pd.DataFrame(ann_list, columns=cols)

In [285]:
prova = load_obj('prova')
prova

,title,price,locali,superficie,bagni,piani,description
0,Nuovi Appartamenti e Attici / Mansarde in Vend...,€ 192.000 - € 705.000,1 - 5,46,NA,NA,\n Le residenze...
1,"Bilocale viale Italo Calvino, Fonte Ostiense, ...",€ 225.000,2,50,1,NA,\n PAPILLO EUR ...
2,"Quadrilocale via Mario Ridolfi 32, Pisana - Br...",€ 359.000,4,130,3,NA,\n Rif: 18100 -...
3,"Appartamento via dei Gandolfi, Camilluccia - F...",€ 800.000,5+,220,3,NA,\n In strada pr...
4,"Villa via Casalmonferrato, Re di Roma, Roma",€ 3.400.000,5+,500,3+,NA,\n Nel quartier...
5,"Appartamento piazza Cavour, Prati, Roma",€ 950.000,5,170,1,NA,"\n Prati, appar..."
6,"Quadrilocale viale della Grande Muraglia 540, ...",€ 250.000,4,110,2,NA,"\n Eur, apparta..."
7,"Appartamento piazza Del Fante, Mazzini - Delle...",€ 1.090.000,5+,190,2,NA,\n In un contes...
8,"Attico / Mansarda via della Frezza, Piazza del...",€ 2.150.000,4,150,1,NA,\n Nel contesto...
9,"Appartamento via Dandolo, Trastevere, Roma",€ 2.500.000,5+,300,3,NA,\n Nella splend...


In [215]:


locali, superficie, bagni, piani = 'NA','NA','NA','NA'


if not link.startswith('https://www.immobiliare.it'):
    link = mainpage + link
subcontent = requests.get(link)
subsoup = BeautifulSoup(subcontent.text, "html.parser")
infos = subsoup.find('div', class_='im-property__features')

price = infos.find('span', class_='features__price--double')
if price is None:
    price = infos.find('ul', class_='features__price-block').find('li', class_='features__price').span
price = price.getText()

subinfos = infos.find('ul', class_='list-inline list-piped features__list')


for list_item in subinfos.find_all('li'):
    feature = list_item.find('div', class_='features__label')
    
    if feature is not None:
        if feature.getText() == 'locali':
            locali = list_item.find('div').find('span').getText()
        elif feature.getText() == 'superficie':
            superficie = list_item.find('div').find('span').getText()
        elif feature.getText() == 'bagni':
            bagni = list_item.find('div').find('span').getText()
        elif feature.getText() == 'piani':
            piani = list_item.find('div').find('span').getText()

description = subsoup.find('div', class_='left-side')
description = description.find('div', class_='col-xs-12 description-text text-compressed').div.getText()


attr_list=[price, locali, superficie, bagni, piani]

ann_list.append(attr_list)



In [222]:
#infos.find('ul', class_='list-inline list-piped features__list')

In [241]:

ann_list = []

locali, superficie, bagni, piani = 'NA','NA','NA','NA'


if not link.startswith('https://www.immobiliare.it'):
    link = mainpage + link
subcontent = requests.get(link)
subsoup = BeautifulSoup(subcontent.text, "html.parser")
infos = subsoup.find('div', class_='im-property__features')

price = infos.find('span', class_='features__price--double')
if price is None:
    price = infos.find('ul', class_='features__price-block').find('li', class_='features__price').span
price = price.getText()

subinfos = infos.find('ul', class_='list-inline list-piped features__list')


for list_item in subinfos.find_all('li'):
    feature = list_item.find('div', class_='features__label')
    
    if feature is not None:
        if feature.getText() == 'locali':
            locali = list_item.find('div').find('span').getText()
        elif feature.getText() == 'superficie':
            superficie = list_item.find('div').find('span').getText()
        elif feature.getText() == 'bagni':
            bagni = list_item.find('div').find('span').getText()
        elif feature.getText() == 'piani':
            piani = list_item.find('div').find('span').getText()

description = subsoup.find('div', class_='left-side')
description = description.find('div', class_='col-xs-12 description-text text-compressed').div.getText()


attr_list=[price, locali, superficie, bagni, piani]

ann_list.append(attr_list)

In [242]:
attr_lst = []

'\n                                box al piano secondo seminterrato edificio A del complesso Levante di 44 mq in ottimo stato vendesi\n            '

In [221]:
print(locali)
print(superficie)
print(bagni)
print(piani)

NA
44
NA
NA
